<a href="https://colab.research.google.com/github/shredder67/text-graph/blob/master/MPNet_main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shredder67/text-graph.git
!wget https://raw.githubusercontent.com/uhh-lt/TextGraphs17-shared-task/e0d52c21ac608e969bfe3ff9c3ac702ba1f5a28d/data/tsv/test.tsv --directory-prefix ./text-graph/data/tsv
!wget https://raw.githubusercontent.com/uhh-lt/TextGraphs17-shared-task/main/data/tsv/train.tsv --directory-prefix ./text-graph/data/tsv

Cloning into 'text-graph'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 213 (delta 12), reused 0 (delta 0), pack-reused 183
Receiving objects: 100% (213/213), 2.06 MiB | 6.79 MiB/s, done.
Resolving deltas: 100% (92/92), done.
--2024-05-18 17:04:37--  https://raw.githubusercontent.com/uhh-lt/TextGraphs17-shared-task/e0d52c21ac608e969bfe3ff9c3ac702ba1f5a28d/data/tsv/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9071510 (8.7M) [text/plain]
Saving to: ‘./text-graph/data/tsv/test.tsv’

test.tsv            100%[===================>]   8.65M  --.-KB/s    in 0.1s    

2024-05-18 17:04:38 (89.6 MB/s) - ‘./text-graph/data/tsv/test.tsv’ saved [9071510/90

In [2]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import os
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoModel, T5ForConditionalGeneration
import torch.optim as optim
from sklearn.metrics import precision_score, f1_score, recall_score
from tqdm import tqdm

In [4]:
SEED = 42

torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float32


## Data Preproc

In [5]:
os.chdir('text-graph')

train_path = './data/tsv/train.tsv'
test_path = './data/tsv/test.tsv'

from data.dataset import TextGraphDataset

## Model prep and finetuning

In [6]:
# %pip install accelerate bitsandbytes

In [7]:
# Load model directly

model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pretrained_bert = AutoModel.from_pretrained(model_name,)

# model_name = "sentence-transformers/all-MiniLM-L6-v2"
# model_name = "whaleloops/phrase-bert"
# model_name = "DeepPavlov/t5-wikidata5M-with-neighbors"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
class QuestionClassifier(nn.Module):
    def __init__(self, pretrained_bert):
        super().__init__()
        self.bert_backbone = pretrained_bert
        self.hidden_size = pretrained_bert.config.hidden_size
        self.head = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size // 2),
            nn.ELU(),
            nn.Linear(self.hidden_size // 2, 1)
        )

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert_backbone(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs.last_hidden_state  # Access the last hidden states
        pooled_output = last_hidden_state[:, 0, :]  # Take the [CLS] token representation
        logits = self.head(pooled_output)
        return logits

model = QuestionClassifier(
    pretrained_bert
).to(DEVICE)


In [10]:
from peft import LoraConfig, LoraModel, get_peft_model

LORA_RANK=16 # 16 default
LORA_ALPHA=32.
LORA_DROPOUT=1e-1

config = LoraConfig(
    task_type="SEQ_CLS",
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    # target_modules=["query", "value"], # for minilm
    target_modules=["q", "v"], # for MPNet
    # target_modules=["q", "v"], # T5 ["q", "v", "k", "o"]
    lora_dropout=LORA_DROPOUT,
    use_rslora=True,
)

lora_model = LoraModel(model, config, "default")
##  make sure to first wrap the base model by calling get_peft_model before wrapping it in PyTorch
# lora_model = get_peft_model(model, config)

print_trainable_parameters(lora_model)

print('Unfreezing head')
# Unfreeze the clf head
for p in lora_model.head.parameters():
    p.requires_grad = True
print_trainable_parameters(lora_model)


trainable params: 589824 || all params: 110371969 || trainable%: 0.5343965549803682
Unfreezing head
trainable params: 885505 || all params: 110371969 || trainable%: 0.8022915673453284


In [11]:
# See if the parameters are frozen

# for n, p in lora_model.named_parameters():
#     print(p.requires_grad, n)

### Functions for training

In [22]:
def train_epoch(model, loader, optimizer, loss_fn, scaler):
    model.train()

    avg_loss = 0.

    predictions = []
    true_labels = []

    for i, batch in tqdm(enumerate(loader), total=len(loader)):

        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(DEVICE)
        # token_type_ids = batch["token_type_ids"].to(DEVICE) # not for T5 and MPNet
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE).float()

        # https://pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            # logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
            logits = model(input_ids=input_ids, attention_mask=attention_mask).squeeze() # for T5 and MPNet
            loss = loss_fn(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # logits = model(
        #     input_ids=input_ids,
        #     attention_mask=attention_mask,
        #     # token_type_ids=token_type_ids
        #     ).squeeze()
        # loss = loss_fn(logits, labels)
        # loss.backward()
        # optimizer.step()

        avg_loss += loss.item()
        with torch.no_grad():
            preds = F.sigmoid(logits).detach().cpu().float().numpy()
            preds = (preds > 0.5) * 1
            y_true = labels.detach().cpu().numpy()

            predictions += preds.tolist()
            true_labels += y_true.tolist()

    avg_loss /= len(loader) + 1
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)

    return avg_loss, f1, precision, recall


@torch.no_grad()
def eval_epoch(model, loader, loss_fn):
    model.eval()

    avg_loss = 0.
    predictions, true_labels = [], []

    for i, batch in tqdm(enumerate(loader), total=len(loader)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        # token_type_ids = batch["token_type_ids"].to(DEVICE)
        labels = batch["labels"].to(DEVICE).float()

        # logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
        logits = model(input_ids=input_ids, attention_mask=attention_mask).squeeze() # for T5 and mpnet
        loss = loss_fn(logits, labels)

        avg_loss += loss.item()
        preds = F.sigmoid(logits).detach().cpu().numpy()
        preds = (preds > 0.5) * 1
        y_true = labels.detach().cpu().numpy()
        predictions += preds.tolist()
        true_labels += y_true.tolist()

    avg_loss /= len(loader)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)

    return avg_loss, f1, precision, recall


def train(model, train_loader, val_loader, optimizer, loss_fn, save_dir, epochs=10, ):
    """_summary_

    Args:
        model (_type_): Moder for training
        train_loader (_type_): _description_
        val_loader (_type_): _description_
        optimizer (_type_): _description_
        loss_fn (_type_): _description_
        save_dir (str): folder name to save ckpt to
        epochs (int, optional): _description_. Defaults to 10.
    """
    scaler = torch.cuda.amp.GradScaler()

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    best_f1_val = 0
    for e in range(epochs):
        loss, f1, prec, rec = train_epoch(model, train_loader, optimizer, loss_fn, scaler=scaler)
        print(f"Train epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.3f}, precision: {prec:.3f}, recall: {rec:.3f}")

        loss, f1, prec, rec = eval_epoch(model, val_loader, loss_fn)
        print(f"Eval epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.3f}, precision: {prec:.3f}, recall: {rec:.3f}")

        if f1 > best_f1_val:
            best_f1_val = f1
            torch.save(
                model.state_dict(),
                os.path.join(save_dir,"minilm--lora-fixed_oversampling-includegraphs-BEST.pth")
            )

        torch.save(
            model.state_dict(),
            os.path.join(save_dir,"minilm--lora-fixed_oversampling-includegraphs-LAST.pth")

            )

## Training, evaluation and submit

In [23]:
def split_train_dev_test(df: pd.DataFrame):
        all_questions = list(df["question"].unique())
        num_questions = len(all_questions)
        random.shuffle(all_questions)

        train_dev_ratio = 0.8
        train_ratio = 0.9
        num_train_dev_questions = int(num_questions * train_dev_ratio)
        train_dev_questions = all_questions[:num_train_dev_questions]
        test_questions = set(all_questions[num_train_dev_questions:])

        num_train_questions = int(len(train_dev_questions) * train_ratio)
        train_questions = set(train_dev_questions[:num_train_questions])
        dev_questions = set(train_dev_questions[num_train_questions:])

        train_df = df[df["question"].isin(train_questions)]
        dev_df = df[df["question"].isin(dev_questions)]
        test_df = df[df["question"].isin(test_questions)]

        return train_df, dev_df, test_df

def split_train_dev(df: pd.DataFrame):
        all_questions = list(df["question"].unique())
        num_questions = len(all_questions)
        random.shuffle(all_questions)

        train_dev_ratio = 0.8
        train_ratio = 0.9
        num_train_questions = int(num_questions * train_dev_ratio)
        train_questions = set(all_questions[:num_train_questions])
        dev_questions = set(all_questions[num_train_questions:])

        train_df = df[df["question"].isin(train_questions)]
        dev_df = df[df["question"].isin(dev_questions)]

        return train_df, dev_df

In [24]:
BATCH_SIZE=32
MAX_LENGTH=200
EPOCHS=20
LR = 1e-4

INCLUDE_GRAPH = True

from sklearn.utils.class_weight import compute_sample_weight
from torch.utils.data import WeightedRandomSampler

# Loading dataframe for making splits
df = pd.read_csv(train_path, sep='\t')
df["label"] = df["correct"].astype(np.float32)
# df_train, df_dev, df_test = split_train_dev_test(df)
df_train, df_dev = split_train_dev(df)

train_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
                            split='train',
                            df_split=df_train,
                            include_graph=INCLUDE_GRAPH,
                            is_T5=False,
                           )
dev_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
                          split='val',
                          df_split=df_dev,
                          include_graph=INCLUDE_GRAPH,
                          is_T5=False,
                         )
# test_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
#                            split='test',
#                            df_split=df_test,
#                            include_graph=INCLUDE_GRAPH,
#                            is_T5=True,
#                           )

weights = compute_sample_weight('balanced', train_ds.labels)
sampler = WeightedRandomSampler(weights, len(weights)) # we will oversample correct answers :)

CONFIG_DATALOADER = {"num_workers":4, "pin_memory":True}
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    sampler=sampler, # does not allow to use shuffle
#     shuffle=True,
    **CONFIG_DATALOADER
)
dev_loader = DataLoader(dev_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, **CONFIG_DATALOADER)
# test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, **CONFIG_DATALOADER)

/content/text-graph/data/dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["graph"] = self.df["graph"].apply(eval)
/content/text-graph/data/dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["graph"] = self.df["graph"].apply(eval)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoa

In [25]:
tokenizer.decode(train_ds[23]['input_ids'], skip_special_tokens=False)

'<s> battle of the sexes, billie jean king : whom did billy jean king beat in the 1973 battle of the sexes tennis match? [ sep ] united states, country, united states ; billie jean king, country of citizenship, united states ; billie jean king, sport, tennis ; [ sep ] rosemary casals [ sep ], country of citizenship, united states ; [ sep ] rosemary casals [ sep ], sport, tennis ; battle of the sexes, sport, tennis ; </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [26]:
tokenizer.decode(dev_ds[0]['input_ids'], skip_special_tokens=False)

'<s> world war i : whose assassination sparked the beginning of world war i? [ sep ] world war i, has immediate cause, [ sep ] assassination of archduke franz ferdinand [ sep ] ; [ sep ] assassination of archduke franz ferdinand [ sep ], immediate cause of, world war i ; </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [27]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=lora_model.parameters(), lr=LR)

In [28]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

# MPNet training

In [30]:
%%time

SAVE_DIR = f"logs-mpnet-cl{MAX_LENGTH}-lingraph_{INCLUDE_GRAPH}-Adam"

train(
    lora_model,
    train_loader,
    dev_loader,
    optimizer,
    loss_fn,
    epochs=EPOCHS,
    save_dir=SAVE_DIR,
)

100%|██████████| 940/940 [04:27<00:00,  3.51it/s]

Train epoch 1 - loss: 0.297, f1: 0.855, precision: 0.824, recall: 0.890



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 238/238 [01:25<00:00,  3.11it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████

Eval epoch 1 - loss: 0.286, f1: 0.528, precision: 0.388, recall: 0.826
CPU times: user 5min 25s, sys: 10.3 s, total: 5min 35s
Wall time: 6min 1s


# Submission

In [31]:
@torch.no_grad()
def make_submit_predictions(model, tokenizer, include_graph, filename='submission.csv'):
    model.eval()
    eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH,  train_path=train_path, test_path=test_path,
                               split='eval', include_graph=include_graph)
    preds = []
    for idx, data in tqdm(enumerate(eval_ds)):
        input_ids = data["input_ids"].to(DEVICE).unsqueeze(0)
        attention_mask = data["attention_mask"].to(DEVICE).unsqueeze(0)
        token_type_ids = data["token_type_ids"].to(DEVICE).unsqueeze(0)

        logit = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
        pred = (logit.detach().cpu().numpy() > 0) * 1
        preds.append(pred)

    df = eval_ds.df
    df['prediction'] = preds
    df['prediction'] = df['prediction'].astype(int)
    df[["sample_id", "prediction"]].to_csv(filename, sep='\t', index=False)

@torch.no_grad()
def make_submit_predictions_ranked(model, tokenizer, include_graph, filename='submission.csv', is_t5=False):
    """based of Vika's idea - select all candidate answers for questions, select one with max prob"""
    model.eval()
    eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH,  train_path=train_path, test_path=test_path,
                               split='eval',
                               df_split=None,
                               include_graph=include_graph)
    eval_df = eval_ds.df
    eval_df["correct"] = False

    for question in tqdm(eval_df['question'].unique()):
        ids = eval_df.index[eval_df['question'] == question].tolist()

        logits = []
        for idx in ids:
            data = eval_ds[idx]
            input_ids = data["input_ids"].to(DEVICE).unsqueeze(0)
            attention_mask = data["attention_mask"].to(DEVICE).unsqueeze(0)
            if is_t5:
                logit = model(input_ids=input_ids, attention_mask=attention_mask,).squeeze()
            else:
                token_type_ids = data["token_type_ids"].to(DEVICE).unsqueeze(0)
                logit = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()


            logits.append(logit.detach().cpu().item())

        right_ans_id = ids[np.argmax(logits)]
        eval_df.loc[right_ans_id, 'correct'] = True

    eval_df['prediction'] = eval_df['correct']
    eval_df['prediction'] = eval_df['prediction'].astype(int)
    eval_df[["sample_id", "prediction"]].to_csv(filename, sep='\t', index=False)

In [40]:
import copy
from pathlib import Path

load_path_best = Path(SAVE_DIR,'minilm--lora-fixed_oversampling-includegraphs-BEST.pth')
load_path_last = Path(SAVE_DIR,'minilm--lora-fixed_oversampling-includegraphs-LAST.pth')

# Best model submission
submission_lora_model = copy.deepcopy(lora_model)
state_dict_loaded = torch.load(load_path_best)
submission_lora_model.load_state_dict(state_dict_loaded)
make_submit_predictions_ranked(
    submission_lora_model,
    tokenizer,
    include_graph=INCLUDE_GRAPH,
    filename=f"{SAVE_DIR}/{SAVE_DIR}-submission_best.csv",
    is_t5=True
)

# Last model submission
submission_lora_model = copy.deepcopy(lora_model)
state_dict_loaded = torch.load(load_path_last)
submission_lora_model.load_state_dict(state_dict_loaded)
make_submit_predictions_ranked(
    submission_lora_model,
    tokenizer,
    include_graph=INCLUDE_GRAPH,
    filename=f"{SAVE_DIR}/{SAVE_DIR}-submission_last.csv",
    is_t5=True
)

100%|██████████| 1000/1000 [03:40<00:00,  4.54it/s]
